In [9]:
from IngestDrugSynonyms import IngestDrugSynonyms
from DrugSynonymDataToNeo4j import DrugSynonymDataToNeo4j

In [10]:
import pandas as pd
import timeit

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/home/ben/Projects/Graphs4GoodHackathon/ProjectDomino/.Domino/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
drugSynonym = IngestDrugSynonyms()
drugSynonym.auto_get_and_clean_data()
drugSynonym.create_drug_study_links()
drugSynonym.create_url_study_links()



2020-04-10 19:26:18,043 - > STARTING scraping with 'covid-19' keyword
2020-04-10 19:26:23,858 - > 427 studies found by 'covid-19' keyword
2020-04-10 19:26:49,102 - > STARTING scraping with 'SARS-CoV-2' keyword
2020-04-10 19:27:08,173 - > 427 studies found by 'SARS-CoV-2' keyword
2020-04-10 19:27:38,606 - > STARTING scraping with 'coronavirus' keyword
2020-04-10 19:27:43,681 - > 346 studies found by 'coronavirus' keyword
2020-04-10 19:28:09,761 - > 1076 distinct studies found
2020-04-10 19:28:12,701 - > 2145 drug & synonym to study connections found


In [ ]:
neo4jBridge = DrugSynonymDataToNeo4j()
neo4jBridge.merge_drugs_synonyms_and_link_between(drugSynonym.drug_vocab)
neo4jBridge.merge_studies(drugSynonym.all_studies_df)

In [ ]:
neo4jBridge.merge_drug_to_study_rels(drugSynonym.appeared_in_edges)

In [ ]:
neo4jBridge.merge_url(list(drugSynonym.all_studies_df["study_url"]))

In [ ]:
neo4jBridge.merge_url_to_study_rels(drugSynonym.url_points_at_study_edges)

In [ ]:
neo4jBridge.drug_or_synonym_name_and_neo4j_id_pairs["Sargramostim"]

In [ ]:
list(map(lambda x: x.lower().strip(),[" A","B ","C"]))

In [ ]:
drugSynonym._filterData()

In [ ]:
temp = [" A","B ","C"]
row = "a"



In [ ]:
list(map(lambda x: x.lower().strip(),temp))

In [ ]:
list(filter(lambda x: x is not None,list(map(lambda x: x.lower().strip() if x.lower().strip() != row['Common name'].lower().strip() else None,row["Synonyms"].split("|")) if isinstance(row["Synonyms"],str) else row["Synonyms"])


In [ ]:
list(filter(lambda x: x is not None,list(map(lambda x: x.lower().strip() if x.lower().strip() != row.lower().strip() else None,temp) ) ))

In [ ]:
drugSynonym.drug_vocab.keys()

In [14]:
drugSynonym.drug_vocab["leucine"]

['(2s)-2-amino-4-methylpentanoic acid',
 '(2s)-alpha-2-amino-4-methylvaleric acid',
 '(2s)-alpha-leucine',
 '(s)-(+)-leucine',
 '(s)-leucine',
 '2-amino-4-methylvaleric acid',
 'l-leucin',
 'l-leucine',
 'l-leuzin',
 'leu']

In [15]:
drugSynonym.save_data_to_file()

2020-04-10 19:32:45,661 - Only Use it for debug purposes!!!


In [16]:
self = drugSynonym

In [29]:
import re
drug_vocab = self.drug_vocab


drugs_and_syms:list = list(drug_vocab.keys())
drugs_and_syms.extend( item.lower() for key in drug_vocab.keys() if isinstance(drug_vocab[key],list) for item in drug_vocab[key] )
ids_and_interventions:list = [(row["trial_id"],row["intervention"].lower()) for row in self.all_studies_df.to_dict('records')]

self.appeared_in_edges:list = [(drug,trial_id) for drug in drugs_and_syms for trial_id,intervention in ids_and_interventions if bool(re.compile(r"\b%s\b" % re.escape(drug)).search(intervention))]


In [27]:
word = 'two'
k = re.compile(r'\b%s\b' % word)
x = 'one two three'
bool(k.search( x))

True

In [30]:
len(self.appeared_in_edges)

435

In [31]:
drug_vocab = self.drug_vocab


drugs_and_syms:list = list(drug_vocab.keys())
drugs_and_syms.extend( item.lower() for key in drug_vocab.keys() if isinstance(drug_vocab[key],list) for item in drug_vocab[key] )
ids_and_interventions:list = [(row["trial_id"],row["intervention"].lower()) for row in self.all_studies_df.to_dict('records')]

self.appeared_in_edges_fast:list = [(drug,trial_id) for drug in drugs_and_syms for trial_id,intervention in ids_and_interventions if drug in intervention]

In [32]:
len(self.appeared_in_edges_fast)

2145